In [1]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud

from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections

from datetime import datetime

import os
from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

import gspread
from oauth2client.service_account import ServiceAccountCredentials

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

In [2]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_amigocloud(ruta_completa):
    try:
        with open(ruta_completa + r'\_keys\api_amigocloud_gral.key', 'r') as acceso:
            api_token = acceso.readline().strip()
        amigocloud = AmigoCloud(token=api_token)
        return amigocloud
    except FileNotFoundError:
        print("El archivo 'api_amigocloud.key' no se encuentra en la ruta especificada.")
    return None

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../unidad_local.txt')
ruta = ruta.readlines()[0]

unidad = get_unidad_local()
ruta_ = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta_)
ruta_completa

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
path_file_gapi = unidad + r'\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\_keys\client_secret_google_sheets.json'
path_file_gapi

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON\\_keys\\client_secret_google_sheets.json'

In [5]:
def crear_cite(cod_canhero, nom_canhero, tipo_reporte):
    # Configura las credenciales
    scope = ["https://spreadsheets.google.com/feeds",
             'https://www.googleapis.com/auth/spreadsheets', 
             "https://www.googleapis.com/auth/drive.file", 
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(path_file_gapi, scope)
    client = gspread.authorize(creds)
    # Abre la hoja de cálculo con ID y la hoja específica
    spreadsheet = client.open_by_key("1Wbmmjy9s8JdXKP8qM_eA_OrCV--F1UZLnzQJ--ASOFI")
    sheet = spreadsheet.sheet1
    # Lee los datos
    data_sheet = sheet.get_all_records()
    df = pd.DataFrame(data_sheet)
    # Encuentra el número más grande y lo incrementamos y agrega un nuevo registro
    max_value = df['No'].max()
    no = int(max_value + 1)
    # llenamos los demas campos
    fecha = datetime.now().strftime("%d/%m/%Y")
    cod_ca = int(cod_canhero)
    nom_ca = nom_canhero
    tipo = tipo_reporte
    #agregar el registro
    sheet.append_row([no, fecha, cod_ca, nom_ca, tipo])
    return no

In [6]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_307560 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = datetime.strptime(credito_data['fecha_3ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito_data['fecha_3ra_insp'] = datetime.strptime(credito_data['fecha_3ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')

    cite = crear_cite(credito_data['canhero'].split(' / ')[0], credito_data['canhero'].split(' / ')[1], 'CRS')
    credito_data['cite_3ra_insp'] = cite
    
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    # query_propiedades = {'query': 'select * from dataset_282968 where cre_ref_prop = \'{cre_ref_prop}\''.format(cre_ref_prop=credito.amigo_id)}
    query_propiedades = {'query': 'select prop.propiedad, prop.amigo_id from dataset_307562 prop inner join dataset_307563 lotes on lotes.prop_ref_lote=prop.amigo_id where insp_ref_prop=\'{cre_ref_prop}\' and (lotes.estado_de_siembra=\'CAÑA NACIDA\' or lotes.estado_de_siembra=\'CAÑA NO NACIDA\') group by prop.propiedad, prop.amigo_id'.format(cre_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_propiedades)    
    
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': 'select * from dataset_307563 where prop_ref_lote=\'{prop_ref_lote}\' and (estado_de_siembra=\'CAÑA NACIDA\' or estado_de_siembra=\'CAÑA NO NACIDA\')'.format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_58438 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))


    # GENERAR REPORTE
    doc = DocxTemplate(ruta + "/templates/tpl_inf_credito_3ra.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open(ruta + '/fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_2da_insp == 'Juan Pablo Jaldin Duran':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_jaldin.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_rogelio.png', width=Mm(60))
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    cod_ca = credito.canhero.split(' / ')[0]
    nom_ca = credito.canhero.split(' / ')[1]
    file_name = file_name = '_' + cod_ca + '_CRS_' + credito.fecha_3ra_insp.replace('/', '-') + '_' + nom_ca + '_' + str(credito.id)
    doc.save(ruta + '/informes/' + file_name + '.docx')

In [7]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]
    
    
    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)
    print(data.crs.name)
    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    ruta_plano = ruta + '/planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(ruta_plano, dpi = 300, bbox_inches='tight')
    plt.clf()
    return ruta_plano

In [9]:
semana = 'S35'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_307560 where semana_3ra = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32672/sql'
select = amigocloud.get(project, query)
data = select['data']

In [10]:
data

[{'cod': '18358', 'nom': 'ROJAS AMELUNGE CARLOS EDUARDO'}]

In [11]:
# lista = [430, 40270, 13132, 874, 41966, 11798, 41544, 14993, 644, 1068, 13763, 2220, 1858, 2719]
lista = [i['cod'] for i in data]

In [12]:
lista

['18358']

In [19]:
lista = [
 '41944',
 '42347',
 '660',
 '40250']

In [13]:
for i in lista:
    generar_reporte(i)
    print('======================', i)

L16 6b8f327250f740c58255149257d25a78
L9.3 11363002d92846dda9c0155b0a429fdc
L3 b5b875a0476b491693373bd676aaee4c
L18 e2566c856a244fa9b33e3a28daac55e0
L4 5589b69015ae43dea97aa3d95925beec
L5 e8cc55d5e2bf4206a2ef673ba1919ae6
L13 4be8abe7feb1410d80c18f4f0f421ea3
L1.3 59759fb2f8114800b60bcda8ede90b85
L1.1 a99ea472a5bf46b0aa68573d3d97991c
L6.1 ce0afa1c1be84c0989dd0a4ac96e5e74
L17 12f64430312243aa8e1998b9209e09ff
L23.2 9c927cef9519404d90b917116f7d2d0e
L30.9 7eaefd0ccee44ff58dc695c97b5a9c22
L19.1 3a39726636fc4b869b9837ddb9d5e50c
L19.2 5fca0dbb8f3e4135939bdadccc538a41
L25.2 1bc13f035d314de494ba37fb72975bea
L20.4 63ed0948d85b43008b8937a61baf1b5a
L24.1 186d517560494b5ca4c2db8eaf6da25c
L12.1 5c1c46dbfaaf4c1c9f2a4c38d3da3f7e
L30.4 db3f153cf0a74e27bbde7ce0158f3c10
L27.1 62188c9632b7406b9af67513d0061dca
L20.1 9af5efa612d44975a9abde131de9b49a
L23.1 2b00116b7f0a49ba9559d67766b671f5
L24.2 89f9e711e03c4abf83f6b73e0e76eaa2
L29.5 f260ee9da17d46a9a646bf57b66632bc
WGS 84 / Pseudo-Mercator
WGS 84 / Pseudo-Merca

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>